In [1]:
from llama_index.core import VectorStoreIndex
from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore

class IngestEvent(Event):
    index: VectorStoreIndex
    
class RetrieverEvent(Event):
    nodes: list[NodeWithScore]

class RerankEvent(Event):
    nodes: list[NodeWithScore]

In [2]:
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.core.postprocessor.llm_rerank import LLMRerank
from llama_index.core.workflow import Context, Workflow, StartEvent, StopEvent, step

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.readers.wikipedia import WikipediaReader

import wikipedia
from wikipedia import PageError

class RAGWorkflow(Workflow):
    @step
    async def ingest(self, ctx: Context, ev: StartEvent) -> IngestEvent | None:
        query = ev.get("query", None)
        if query is None:
            return None

        await ctx.set("query", query)

        pages = wikipedia.search(query, results=10)
        if not pages:
            return None

        wiki_loader = WikipediaReader()
        documents = []
        for page in pages:
            try:
                doc = wiki_loader.load_data([page], lang_prefix="en")
                documents.extend(doc)
            except PageError:
                print(f"Skipping “{page}” (PageError).")

        index = VectorStoreIndex.from_documents(
            documents=documents,
            embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
        )

        return IngestEvent(index=index)

    @step
    async def retrieve(self, ctx: Context, ev: IngestEvent) -> RetrieverEvent | None:
        index = ev.index
        query = await ctx.get("query", None)
        if (query is None) or (index is None):
            return None

        retriever = index.as_retriever(similarity_top_k=5)
        nodes = await retriever.aretrieve(query)
        print(f"Retrieved {len(nodes)} nodes.")
        return RetrieverEvent(nodes=nodes)

    @step
    async def rerank(self, ctx: Context, ev: RetrieverEvent) -> RerankEvent:
        ranker = LLMRerank(
            choice_batch_size=5,
            top_n=3,
            llm=OpenAI(model="gpt-4o"),
        )
        query = await ctx.get("query", default=None)
        new_nodes = ranker.postprocess_nodes(ev.nodes, query_str=query)
        print(f"Reranked nodes to {len(new_nodes)}")
        return RerankEvent(nodes=new_nodes)

    @step
    async def synthesize(self, ctx: Context, ev: RerankEvent) -> StopEvent:
        llm = OpenAI(model="gpt-4o")
        summarizer = CompactAndRefine(llm=llm, streaming=True, verbose=True)
        query = await ctx.get("query", default=None)
        
        response = await summarizer.asynthesize(query, nodes=ev.nodes)
        return StopEvent(result=response)

In [3]:
w = RAGWorkflow(timeout=60, verbose=True)

result = await w.run(query="What is a transformer, and how is it used in large language models?")
async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Running step ingest
Skipping “GPT-4” (PageError).
Skipping “GPT-3” (PageError).
Step ingest produced event IngestEvent
Running step retrieve
Retrieved 5 nodes.
Step retrieve produced event RetrieverEvent
Running step rerank
Reranked nodes to 3
Step rerank produced event RerankEvent
Running step synthesize
Step synthesize produced event StopEvent
A transformer is a deep learning architecture that utilizes a multi-head attention mechanism to process and contextualize text data. It converts text into numerical representations called tokens, which are then transformed into vectors using a word embedding table. The architecture allows for parallel processing of tokens, enhancing the importance of key tokens while diminishing less important ones. This design eliminates the need for recurrent units, making transformers more efficient in training compared to earlier recurrent neural networks like LSTMs.

In large language models (LLMs), transformers are used to handle vast amounts of text data

In [4]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(RAGWorkflow, filename="wikipedia_rag_workflow.html")

wikipedia_rag_workflow.html
